In [1]:
%load_ext autoreload
%autoreload 2

from config.rnn import default
from models import RNN
import numpy as np
from functional import seq
import tensorflow as tf
from tensorpack import (TrainConfig, SyncMultiGPUTrainerParameterServer as Trainer, 
                        PredictConfig, MultiProcessDatasetPredictor as Predictor,
                        SaverRestore, logger)
from tensorpack.callbacks import (ScheduledHyperParamSetter, MaxSaver, ModelSaver,
                                  DataParallelInferenceRunner as InfRunner)
from tensorpack.predict import SimpleDatasetPredictor
from tensorpack.tfutils.common import get_default_sess_config
from utils import DataManager, select_label
from utils.validation import (Accumulator, AggregateMetric, calcu_metrics)

resnet_loc = "./data/resnet_v2_101/resnet_v2_101.ckpt"
model_loc = "./train_log/separate/read3-noglps-lmd-0/all-stages-max-micro-auc.tfmodel"
log_dir = './train_log/separate/D1-read3-mtd/'
logger.set_logger_dir(log_dir)

/home/zhoumingyu/bin/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/zhoumingyu/bin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[1103 20:40:27 @logger.py:74] Argv: /home/zhoumingyu/bin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1017/jupyter/kernel-edecef4c-c133-4c0e-b4cc-4b488903a73c.json


In [2]:
config = default
ignore_restore = ['learning_rate', 'global_step']
save_name = "max-macro_f1.tfmodel"

In [3]:
config.stages = [2, 3, 4, 5, 6]
config.proportion = {'train': 0.55, 'val': 0.0, 'test': 0.45}
config.annotation_number = None
dm = DataManager.from_config(config)
train_set = dm.get_train_set()
test_set = dm.get_test_set()

In [4]:
config.proportion = {'train': 0.8, 'val':0.2, 'test': 0.0}
config.stages = [2]
config.annotation_number = 10
dm = DataManager.from_dataset(train_set, test_set, config)
dm.get_num_info()

{'test': (762, 3237), 'train': (692, 3016), 'val': (156, 572)}

In [5]:
dm.get_imbalance_ratio()

,train,val,test
anlage in statu nascendi,7.439024,7.666667,7.758621
cellular blastoderm,1.005797,1.025974,1.005263
dorsal ectoderm anlage in statu nascendi,6.440860,6.090909,6.398058
faint ubiquitous,5.653846,5.782609,4.953125
maternal,3.907801,4.379310,4.013158
posterior endoderm anlage in statu nascendi,15.878049,14.600000,15.565217
procephalic ectoderm anlage in statu nascendi,7.650000,7.666667,7.758621
subset,3.552632,3.457143,3.590361
ubiquitous,0.870270,0.714286,0.809976
ventral ectoderm anlage in statu nascendi,7.650000,6.800000,7.282609


In [ ]:
config.weight_decay = 0.0
config.dropout_keep_prob = 0.4
config.gamma = 2
config.use_glimpse = False
config.use_hidden_dense = False
config.read_time = 3
config.batch_size = 64

threshold = 0.4
train_data = dm.get_train_stream()
val_data = dm.get_validation_stream()
#val_data = dm.get_test_stream()

In [ ]:
np.set_printoptions(formatter={'float_kind': lambda x: '%.2f' % x})
model = RNN(config, is_finetuning=False, label_weights=dm.get_imbalance_ratio().train.values)
tf.reset_default_graph()
train_config = TrainConfig(model=model, dataflow=train_data,
                           callbacks=[
                               ScheduledHyperParamSetter('learning_rate', [(0, 1e-4),  
                                                                           (40, 1e-5)]),
                               InfRunner(val_data, [AggregateMetric(config.validation_metrics, threshold)],
                                         [1]),
                               ModelSaver(var_collections='model_variables'),
                               MaxSaver('micro_auc', save_name),
                           ],
                           session_init=SaverRestore(
                               model_path=resnet_loc, 
                               ignore=ignore_restore),
                           max_epoch=1, tower=[0])
Trainer(train_config).train()

In [ ]:
tf.get_collection(tf.GraphKeys.MODEL_VARIABLES)

In [ ]:
model = RNN(config, is_finetuning=False)
tf.reset_default_graph()
pred_config = PredictConfig(model=model,
                            session_init=SaverRestore(
                                model_path=log_dir + save_name),
                            output_names=['logits_export', 'label'],
                            )
# pred = Predictor(pred_config, test_data, nr_proc=2, ordered=False)
pred = SimpleDatasetPredictor(pred_config, test_data)

accumulator = seq(pred.get_result()) \
    .smap(lambda a, b: (a.shape[0], calcu_metrics(a, b, config.validation_metrics, threshold))) \
    .aggregate(Accumulator(*config.validation_metrics), lambda accu, args: accu.feed(args[0], *args[1]))
metrics = accumulator.retrive()
print(metrics)